# Benchmark calculcations for January and June proposals

Assumptions:

- Buildings will produce the max amount of emissions, **unless** their predicted emissions is lower than the target GHGI, in which case they will produce that amount.
- Predicted emissions are calculated based off of the emissions predictions found in [the emissions data input file](../data/input_data/energy_emissions.csv). 



For each building, in each year (2027-2050), we need to calculate (column references given refer to the 2027 calculations in [the original RMI spreadsheet](https://docs.google.com/spreadsheets/d/175uipAHHQHGelq7i1n9sKWQXNQi-B1IJiF6-XQRVnE8/edit#gid=1811888818):

- `city_ghgi_target`: The GHGI target, as calculated by the sum of: use type * city's target GHGI for that use type * percent of GFA for that use type, for each of the three given use types (col A). If there is no target for any of the use types, this is NaN. If a building is multi-use, and some of the building's uses have a compliance threshold, but others don't, use the expected GHGI (greenhouse gas emissions intensity) if nothing is changed as the GHGI for the portion of the building that is not subject to BEPS yet.
    - Example: A building is 50% retail and 50% multifamily housing. The target for retail in 2033 is 1.03 and there is no target for multifamily housing. This building would have a GHGI of 4.0 in 2033 if no changes were made to it. We estimate the GHGHI target as `(0.5 * 1.03) + (0.5 * 4.0) = 2.515`. 
    - NB: These numbers will be different than the RMI calculations. The RMI model mistakenly used a GHGI of zero for parts of buildings that are not yet subject to BEPS. So in the example above, RMI's model would list the GHGI target as`0.5 * 1.03 + 0.5 * 0 = 0.515`, which leaves out half the building. Then the target GHGI would actually go **up** when the multifamily part of the building became subject to BEPS! This model corrects that error.
- `expected_baseline`: The expected emissions if nothing is changed about the building, as calculated by the sum of: `total use energy for type * energy emissions factor for energy type` for the three energy types (col C)
- `expected_baseline_ghgi`: The expected GHGI if nothing is changed about the building (col B), as calculated by the `expected emissions / total GFA`
- `compliant_ghgi`: The expected GHGI if the building is compliant with BEPS, as defined by (col H):
    - if the BEPS GHGI target is lower than the expected GHGI, use the BEPS GHGI target
    - if the expected GHGI is lower than the BEPS GHGI target, use the expected GHGI
- `compliant_emissions`: The expected emissions if the building is compliant with BEPS, as defined by the `compliant GHGI * total GFA` (col J)
- `compliance_status`: Whether or not the building is compliant (col K):
    - yes: the baseline GHGI is lower than the expected compliant GHGI for this year
    - no: the baseline GHGI is higher than the expected compliant GHGI for this year
    - no requirement yet: the building doesn't have a compliance requirement for this year
- `compliance_fees`: Noncompliance fees. For years where buildings will be taxed for being noncompliant, this is `$2.50 * total GFA`    

In [1]:
import pandas as pd
import numpy as np

In [2]:
from baseline_model import BaselineBEPSModel

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
JAN_TARGETS_PATH = '../data_new/input_data/jan_proposal_emissions_targets.csv'
JUNE_TARGETS_PATH = '../data_new/input_data/june_proposal_emissions_targets.csv'
EMISSIONS_PATH = '../data_new/input_data/energy_emissions.csv'
BUILDING_DATA_PATH = '../data_new/input_data/cleaned_building_data_with_policy_gfa.csv'

JAN_FINE_YEARS = [2027, 2030, 2035, 2040, 2045, 2050]
JUNE_FINE_YEARS = [2030, 2035, 2040, 2045, 2050]
FINE_PER_SQ_FT = 2.5

## January Proposal

In [5]:
baseline_path = '../data_new/output_data/jan_baseline_scenario.csv'
jan_model = BaselineBEPSModel(EMISSIONS_PATH, JAN_TARGETS_PATH, BUILDING_DATA_PATH, JAN_FINE_YEARS, FINE_PER_SQ_FT, baseline_path)

In [ ]:
jan_model.calculate_baseline_model(2027,2050)

In [ ]:
jan_model.scenario_results.head()

In [ ]:
# remove me
m = pd.read_csv('../data_new/output_data/jan_baseline_scenario.csv')

In [ ]:
# delete me too
jan_model.scenario_results = m

In [6]:
jan_model.scenario_results.head()

,Unnamed: 0,OSEBuildingID,BuildingName,Total GFA for Policy,sq_ft_classification,LargestPropertyUseType OSE,SecondLargestPropertyUseType OSE,ThirdLargestPropertyUseType OSE,year,expected_baseline,expected_baseline_ghgi,city_ghgi_target,compliant_ghgi,compliant_emissions,compliance_status,compliance_fees
0,0,1,MAYFLOWER PARK HOTEL,88434.0,C,Hotel,NaN,NaN,2027.0,2.705014e+05,3.058794,3.058794,3.058794,2.705014e+05,No,221085.0
1,1,2,PARAMOUNT HOTEL,103566.0,B,Hotel,NaN,Restaurant,2027.0,2.843616e+05,2.745704,2.059096,2.059096,2.132523e+05,No,258915.0
2,2,3,WESTIN HOTEL (Parent Building),895128.0,A,Hotel,NaN,Recreation,2027.0,2.193901e+06,2.450936,1.876172,1.876172,1.679414e+06,No,2237820.0
3,3,5,HOTEL MAX,61320.0,C,Hotel,NaN,NaN,2027.0,2.934861e+05,4.786141,4.786141,4.786141,2.934861e+05,No,153300.0
4,4,8,WARWICK SEATTLE HOTEL,191454.0,B,Hotel,NaN,Recreation,2027.0,5.827520e+05,3.043822,1.431403,1.431403,2.740479e+05,No,478635.0


In [ ]:
jan_model.scenario_results.columns

## Summary stats

In [7]:
jan_model.get_total_emissions_by_year()

Emissions by year calculations complete. Access the annual emissions dataframe as model_name.emissions_by_year


In [8]:
jan_model.emissions_by_year

,compliant_emissions
year,
2027.0,3.335484e+08
2028.0,3.225803e+08
2029.0,3.067583e+08
2030.0,2.780274e+08
2031.0,2.462242e+08
2032.0,2.277874e+08
2033.0,2.196453e+08
2034.0,2.100763e+08
2035.0,1.975086e+08


In [9]:
jan_model.get_percent_emissions_reduction_by_given_year(2040)

0.8637739067995087

In [10]:
jan_model.find_reduction_percent(2040, 90, 'percent_reduction')

TypeError: BaselineBEPSModel.find_reduction_percent() takes from 2 to 3 positional arguments but 4 were given